In [16]:
from pyomo.environ import *
import pandas as pd

model = ConcreteModel()

Leemos los datos de las corrientes calientes

In [17]:
Hot_df = pd.read_excel('Datos/Ejercicio3.xlsx', 'Hot',skiprows = 1,index_col=0)
Hot_df

,0,1,2,3,4,5,6,7,8,9
ID,,,,,,,,,,
H1,0,360,60,180,60,180,60,0,0,0
H2,0,0,0,0,0,540,0,0,0,0
H3,0,0,0,240,80,240,80,160,80,160


E igualmente de las frías.

In [18]:
Cold_df = pd.read_excel('Datos/Ejercicio3.xlsx', 'Cold',skiprows = 1,index_col=0)
Cold_df

,0,1,2,3,4,5,6,7,8,9
ID,,,,,,,,,,
C1,0,0,150,450,150,450,150,300,150,0
C2,0,0,0,0,160,480,160,320,0,0


In [19]:
hotst = Hot_df.index.values.tolist()
coldst = Cold_df.index.values.tolist()
s=hotst.copy()
s.insert(0,'S')
w=coldst.copy()
w.insert(0,'W')

model.i=Set(initialize=hotst)
model.j=Set(initialize=coldst)
model.s=Set(initialize=s)
model.S=Set(initialize='S')
model.w=Set(initialize=w)
model.W=Set(initialize='W')
model.k=Set(initialize=Hot_df.columns)


Podemos ver lo que hemos creado hasta ahora haciendo un model.pprint()

In [20]:
model.pprint()

7 Set Declarations
    S : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    1 :  {'S',}
    W : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    1 :  {'W',}
    i : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'H1', 'H2', 'H3'}
    j : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'C1', 'C2'}
    k : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
    s : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'S', 'H1', 'H2', 'H3'}
    w : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : M

Creamos ahora las variables del modelo. Hay i potenciales aportes de calor por utilities calientes (Qs), uno en cada intervalo, al igual que de frías (Qw). De esta forma damos la oportunidad de que entren las utilities en cualquier nivel de temperatura. Creamos también i residuos (R), aunque sabemos que no habrá residuo del intervalo 0 ni tampoco del intervalo 9, lo cual implementaremos como dos restricciones diferentes.

In [21]:
Qs = model.Qs = Var(model.k,within = NonNegativeReals) 
Qw = model.Qw = Var(model.k,within = NonNegativeReals) 
R = model.R = Var(model.s,model.k,within = NonNegativeReals)
Q = model.Q = Var(model.s,model.w,model.k,within = NonNegativeReals)

Buscamos la minimización del vapor del intervalo 1 más la utility fría del intervalo 9.

In [22]:
model.util = Objective(expr = model.Qs[1] + model.Qw[9])

Constraints

In [23]:
nk = list(model.k)[1:] #Creamos una lista que contenga los índices sobre los que haremos el balance. En nuestro caso el balance lo aplicamos en todos los intervalos, excepto el intervalo 0. Es decir sobre 1-5, lo cual expresamos por [1:], es decir, desde 1 hasta el final de la lista.
model.C1 = ConstraintList()
for k in nk:
    for i in model.i:
        model.C1.add(
        R[i,k-1]+Hot_df.loc[i,k] == R[i,k]+sum(Q[i,j,k] for j in model.j)+sum(Q[i,w,k] for w in model.W)
        )

model.C2 = ConstraintList()
for k in nk:
    for s in model.S:
        model.C2.add(
        R[s,k-1]+Qs[k] == R[s,k]+sum(Q[s,j,k] for j in model.j)
        )        

model.C3 = ConstraintList()
for k in nk:
    for j in model.j:
        model.C3.add(
        sum(Q[i,j,k] for i in model.i)+ sum(Q[i,j,k] for i in model.S) == Cold_df.loc[j,k]
        )

model.C4 = ConstraintList()
for k in nk:
        model.C4.add(
        sum(Q[i,'W',k] for i in model.i)  == Qw[k]
        )


model.C5 = ConstraintList()
for s in model.s:
    model.C5.add(
        R[s,0]==0
        )

model.C6 = ConstraintList()
for s in model.s:
    model.C6.add(
        R[s,9]==0
        )

nm = list(model.k)[0:1]+list(model.k)[2:]
model.C7 = ConstraintList()
for m in nm:
    model.C7.add(
        Qs[m]==0
        )

nn = list(model.k)[0:9]
model.C8 = ConstraintList()
for n in nn:
    model.C8.add(
        Qw[n]==0
        )

model.C9 = ConstraintList()
for k in model.k:
    model.C9.add(
        Q['S','W',k] ==0
    )

Resolvemos el modelo

In [24]:
results = SolverFactory('glpk').solve(model)
model.pprint()
results.write()

18 Set Declarations
    C1_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   27 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27}
    C2_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    9 : {1, 2, 3, 4, 5, 6, 7, 8, 9}
    C3_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   18 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18}
    C4_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    9 : {1, 2, 3, 4, 5, 6, 7, 8, 9}
    C5_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    C6_index : Size=1, Index=None, Ordered=Insertion

In [25]:
Qss = value(model.Qs[1])
Qww = value(model.Qw[9])
print('Cold utility = {0:2.2f}, Hot utility = {1:2.2f}'.format(Qww, Qss))

Cold utility = 160.00, Hot utility = 600.00


Ahora resolvemos el problema de mínimo número de intercambios

In [26]:
U = {'C1': [600, 900,540,1040], 'C2': [600,900,540,1040], 'W':[0,160,160,160]}
U_df = pd.DataFrame(U, index=['S', 'H1', 'H2','H3'])
U_df

,C1,C2,W
S,600,600,0
H1,900,900,160
H2,540,540,160
H3,1040,1040,160


In [27]:
model.y=Var(model.s,model.w, within=Binary)

model.C10 = ConstraintList()
for s in model.s:
    for w in model.w:
        model.C10.add(
        sum(Q[s,w,k] for k in model.k)-U_df.loc[s,w]*model.y[s,w] <=0
        )    

In [28]:
model.util.deactivate()
model.sumy = Objective(expr=sum(model.y[s,w] for s in model.s for w in model.w))

In [30]:
results = SolverFactory('glpk').solve(model)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 6.0
  Upper bound: 6.0
  Number of objectives: 1
  Number of constraints: 112
  Number of variables: 193
  Number of nonzeros: 456
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 13
      Number of created subproblems: 13
  Error rc: 0
  Time: 0.04109477996826172
# ----------------------------------------------------------
#   Solution Information
# ---------------------------------